<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220411_sm_SANUP_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-ilz5pa4_
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-ilz5pa4_
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [3]:
from pykospacing import Spacing

In [4]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers
# !pip install git+https://github.com/ssut/py-hanspell.git

     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 4.0 MB 78.5 MB/s 
     |████████████████████████████████| 1.1 MB 82.9 MB/s 
     |████████████████████████████████| 136 kB 79.6 MB/s 
     |████████████████████████████████| 212 kB 84.7 MB/s 
     |████████████████████████████████| 77 kB 6.9 MB/s 
     |████████████████████████████████| 127 kB 85.9 MB/s 
     |████████████████████████████████| 271 kB 82.4 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 86.4 MB/s 
     |████████████████████████████████| 895 kB 74.7 MB/s 
     |████████████████████████████████| 6.5 MB 74.5 MB/s 
     |████████████████████████████████| 596 kB 74.4 MB/s 
     |████████████████████████████████| 1.2 MB 70.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

     |████████████████████████████████| 65 kB 2.6 MB/s 
     |████████████████████████████████| 149.9 MB 34 kB/s 
     |████████████████████████████████| 831.4 MB 2.5 kB/s 
     |████████████████████████████████| 57 kB 5.5 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.11
    Uninstalling google-api-python-client-1.12.11:
      Successfully uninstalled google-api-python-client-1.12.11
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
torchaudio 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher
from pathlib import Path
# from hanspell import spell_checker

- TPU 사용을 위한 accelerate

In [6]:
from accelerate import Accelerator

In [7]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

In [8]:
df = df.fillna('')
df['text'] = df['text_obj'].astype(str) + " " + df['text_mthd'].astype(str) + " " + df['text_deal'].astype(str)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차부분정비 타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반소비자에게 열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학전아동보육
...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,제품입고 워싱 청바지워싱
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,현장에서 고객의요청에의해 실내인테리어
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,영업점에서 일반소비자에게 여성의류 판매
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,사업장에서 일반고객을대상으로 필라테스


# 토크나이저, 함수

In [9]:
def remove_something(text:str):
  return text.replace('&',' ')

df['text'] = df['text'].map(remove_something)

def remove_something(text:str):
  return text.replace('.',' ')

df['text'] = df['text'].map(remove_something)

def remove_something(text:str):
  return text.replace(',',' ')

df['text'] = df['text'].map(remove_something)

In [10]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

In [11]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [12]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [13]:
spacing = Spacing()

def text_space(text:str):
  return spacing(text)

## 띄어쓰기 전처리

In [14]:
for i in tqdm(range(1,11)):
  my_file = Path("/content/drive/MyDrive/220407_inference/train"+str(i)+'.csv')
  if my_file.is_file():
    continue
  start = (i-1)*100000
  end = i*100000
  temp = df[start:end]
  temp['text'] = temp['text'].map(text_space)
  temp.to_csv('/content/drive/MyDrive/220407_inference/train'+str(i)+'.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [15]:
df = pd.read_csv('/content/drive/MyDrive/220407_inference/train1.csv')

for i in tqdm(range(2, 11)):
  temp = pd.read_csv('/content/drive/MyDrive/220407_inference/train'+str(i)+'.csv')
  df = pd.concat([df,temp], axis=0)

  0%|          | 0/9 [00:00<?, ?it/s]

In [16]:
df = df.reset_index(drop=True)

In [17]:
test = test.fillna('')
test['text'] = test['text_obj'].astype(str) + " " + test['text_mthd'].astype(str) + " " + test['text_deal'].astype(str)
test['target'] = test['digit_1'].astype(str) + " " + test['digit_2'].astype(str) + " " + test['digit_3'].astype(str)
test = test[['text', 'target']]

def remove_something(text:str):
  return text.replace('&',' ')

test['text'] = test['text'].map(remove_something)

def remove_something(text:str):
  return text.replace('.',' ')

test['text'] = test['text'].map(remove_something)

def remove_something(text:str):
  return text.replace(',',' ')

test['text'] = test['text'].map(remove_something)

In [18]:
for i in tqdm(range(1,11)):
  my_file = Path("/content/drive/MyDrive/220407_inference/test"+str(i)+'.csv')
  if my_file.is_file():
    continue
  start = (i-1)*10000
  end = i*10000
  temp = test[start:end]
  temp['text'] = temp['text'].map(text_space)
  temp.to_csv('/content/drive/MyDrive/220407_inference/test'+str(i)+'.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [19]:
test = pd.read_csv('/content/drive/MyDrive/220407_inference/test1.csv')

for i in tqdm(range(2, 11)):
  temp = pd.read_csv('/content/drive/MyDrive/220407_inference/test'+str(i)+'.csv')
  test = pd.concat([test,temp], axis=0)

  0%|          | 0/9 [00:00<?, ?it/s]

In [20]:
test = test.reset_index(drop=True)
test_original = test

# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

In [21]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220407_inference/digit'+str(digit)+'/model'+str(fold), save_function=accelerator.save)

## digit 별 학습

In [22]:
for digit in range(1,4):
  my_file = Path('/content/drive/MyDrive/220407_inference/digit'+str(digit)+'/model'+str(4))
  if my_file.is_dir():
    continue
  # my_file = Path("/content/drive/MyDrive/220407_inference/digit"+str(i))
  # if my_file.is_dir():
  #   continue
  print('digit'+str(digit))

  #train
  train = df[['text']]
  target = 'digit_'+str(digit)
  train['target'] = df[target]

  my_dict = {}
  for i in range(len(train['target'].unique())):
    my_dict[train['target'].unique()[i]] = i

  def target_to_num(target:str):
    return my_dict[target]

  train['target'] = train['target'].map(target_to_num)
  my_dict['  '] = len(my_dict)

  kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

  print('--------------------------------')

  tokenized_train = tokenizer(
    list(train['text']),
    return_tensors="pt",
    max_length=64, # Max_Length = 64
    padding=True,
    truncation=True,
    add_special_tokens=True
    )
  
  for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['target'])):
    my_file = Path('/content/drive/MyDrive/220407_inference/digit'+str(digit)+'/model'+str(fold))
    if my_file.is_dir():
      continue
    print(f'FOLD {fold}')
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    train_label = train['target']
    train_dataset = BERTDataset(tokenized_train, train_label)
    notebook_launcher(training_function)

  #test
  test = test_original
  tokenizer = AutoTokenizer.from_pretrained(checkpoint)
  test_label = test['target'].map(target_to_num)
  tokenized_test = tokenizer(
      list(test['text']),
      return_tensors="pt",
      max_length=64,
      padding=True,
      truncation=True,
      add_special_tokens=True
  )

  test_dataset = BERTDataset(tokenized_test, test_label)
  dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)

  for fold in range(5):
    config = AutoConfig.from_pretrained(checkpoint)
    config.num_labels = len(train['target'].unique())
    model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220407_inference/digit'+str(digit)+'/model'+str(fold), config=config)
    model.resize_token_embeddings(tokenizer.vocab_size)
    accelerator = Accelerator()
    model = accelerator.unwrap_model(model)
    output_pred = []
    output_prob = []
    model, dataloader= accelerator.prepare(model, dataloader)
    model.eval()
    for i, data in enumerate(tqdm(dataloader)):
        with torch.no_grad():
            outputs = model(
                input_ids=data['input_ids'],
                attention_mask=data['attention_mask']
            )
        logits = outputs[0]
        prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
        logits = logits.detach().cpu().numpy()
        result = np.argmax(logits, axis=-1)
        output_pred.append(result)
        output_prob.append(prob)
        
    pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
    def num_to_label(label):
        label_dict = {v: k for k, v in my_dict.items()}
        str_label = []
        for i, v in enumerate(label):
            str_label.append([i,label_dict[v]])
        return str_label

    answer = num_to_label(pred_answer)
    df_prob = pd.DataFrame(output_prob)
    df_prob.to_csv('/content/drive/MyDrive/220407_inference/digit'+str(digit)+'/prob_pred'+str(fold)+'.csv', index=False)

## 이 밑에 부분 구현
digit1 - prob_pred0, prob_pred1, prob_pred2, prob_pred3, prob_pred4 -> 소프트 보팅 -> digit1 예측

digit2 - prob_pred0, prob_pred1, prob_pred2, prob_pred3, prob_pred4 -> 소프트 보팅 -> digit2 예측

digit3 - prob_pred0, prob_pred1, prob_pred2, prob_pred3, prob_pred4 -> 소프트 보팅 -> digit3 예측

submission -> digit_1, digit_2, digit_3 -> 예측 값으로 교체 -> 제출

### digit_1(soft voting)

In [23]:
pred0 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit1/prob_pred0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit1/prob_pred1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit1/prob_pred2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit1/prob_pred3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit1/prob_pred4.csv')

In [24]:
pred_1 = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test_1 = pd.concat([test, pred_1], axis=1)
test_1.head(3)

,text,target,0,1,2,3,4,5,6,7,...,9,10,11,12,13,14,15,16,17,18
0,치킨전문점에서 고객의 주문에 의해 치킨 판매,,0.000002,0.000186,0.000002,0.000024,0.999766,5.467185e-07,9.362302e-07,0.000001,...,0.000002,4.796749e-06,0.000001,0.000001,1.623104e-06,1.667585e-07,6.103200e-07,2.645383e-07,3.610479e-07,9.897334e-07
1,산업공구 다른 소매업자에게 철물 수공구,,0.000009,0.999816,0.000001,0.000133,0.000005,9.182562e-08,1.257996e-05,0.000002,...,0.000007,5.643120e-07,0.000001,0.000003,3.701656e-07,1.801013e-07,2.072175e-07,2.411357e-06,2.455445e-07,2.935047e-06
2,절에서 신도을 대상으로 불교단체 운영,,0.999869,0.000029,0.000010,0.000019,0.000004,1.083666e-06,9.530388e-06,0.000007,...,0.000012,1.151846e-05,0.000011,0.000002,1.054538e-05,5.621092e-09,2.785525e-07,1.236651e-06,6.794995e-08,3.977178e-07


In [25]:
train = df[['text']]
target = 'digit_'+str(1)
train['target_1'] = df[target]

my_dict1 = {}
for i in range(len(train['target_1'].unique())):
  my_dict1[train['target_1'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict1[target]

In [26]:
def num_to_label1(label):
    label_dict = label_dict = {v: k for k, v in my_dict1.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label1(np.argmax(np.array(pred_1), axis=-1))

test_1['soft_1'] = pd.DataFrame(answer)[1]

### digit_2

In [27]:
pred0 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit2/prob_pred0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit2/prob_pred1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit2/prob_pred2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit2/prob_pred3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit2/prob_pred4.csv')

In [28]:
pred_2 = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test_2 = pd.concat([test, pred_2], axis=1)
test_2.head(3)

,text,target,0,1,2,3,4,5,6,7,...,64,65,66,67,68,69,70,71,72,73
0,치킨전문점에서 고객의 주문에 의해 치킨 판매,,1.649548e-07,0.000169,0.000031,1.315433e-06,0.000002,0.999718,3.988292e-05,3.101571e-08,...,2.027745e-07,3.921980e-07,9.580889e-07,3.361500e-09,5.828244e-07,1.254408e-08,1.961758e-07,1.222474e-08,2.130103e-09,1.413608e-08
1,산업공구 다른 소매업자에게 철물 수공구,,1.296446e-05,0.005581,0.993394,5.499944e-07,0.000177,0.000005,1.236032e-05,3.921850e-09,...,1.525886e-06,2.430964e-07,5.491305e-06,2.165879e-09,2.737460e-06,7.714892e-08,1.274578e-07,2.826766e-08,1.078178e-08,4.894658e-08
2,절에서 신도을 대상으로 불교단체 운영,,2.065660e-06,0.000011,0.000008,2.268110e-05,0.000005,0.000011,2.569899e-07,9.567682e-06,...,6.939157e-07,4.548492e-07,1.424574e-06,5.290609e-08,1.791180e-07,2.549536e-08,4.901613e-07,6.484129e-08,4.781429e-09,1.538654e-08


In [29]:
train = df[['text']]
target = 'digit_'+str(2)
train['target_2'] = df[target]

my_dict2 = {}
for i in range(len(train['target_2'].unique())):
  my_dict2[train['target_2'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict2[target]

In [30]:
def num_to_label2(label):
    label_dict = label_dict = {v: k for k, v in my_dict2.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label2(np.argmax(np.array(pred_2), axis=-1))

test_2['soft_2'] = pd.DataFrame(answer)[1]
test_2['soft_2'] = test_2['soft_2'].astype(int)

### digit_3

In [31]:
pred0 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit3/prob_pred0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit3/prob_pred1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit3/prob_pred2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit3/prob_pred3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220407_inference/digit3/prob_pred4.csv')

In [32]:
pred_3 = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test_3 = pd.concat([test, pred_3], axis=1)
test_3.head(3)

,text,target,0,1,2,3,4,5,6,7,...,215,216,217,218,219,220,221,222,223,224
0,치킨전문점에서 고객의 주문에 의해 치킨 판매,,3.122445e-08,0.000141,4.663933e-07,7.090262e-07,0.000002,4.434741e-06,9.987078e-01,1.903153e-05,...,7.487947e-09,1.013594e-09,5.619935e-08,4.225392e-09,1.238458e-09,4.874697e-09,3.575370e-10,7.074856e-09,1.749770e-09,3.436565e-09
1,산업공구 다른 소매업자에게 철물 수공구,,5.964648e-07,0.000001,1.015011e-03,3.778387e-03,0.000001,2.229297e-04,1.798778e-06,2.917878e-07,...,5.525362e-08,7.765147e-08,1.334349e-06,2.628872e-08,1.262263e-08,2.191390e-08,2.438576e-08,1.424546e-07,5.996928e-09,1.404645e-08
2,절에서 신도을 대상으로 불교단체 운영,,1.078129e-07,0.000001,2.070131e-07,2.036383e-07,0.000045,5.750106e-07,9.280634e-07,1.654305e-06,...,7.779419e-08,6.600579e-09,5.091248e-08,5.072796e-09,2.062232e-09,5.798554e-08,8.091350e-09,2.715482e-08,1.533450e-09,5.365012e-09


In [33]:
train = df[['text']]
target = 'digit_'+str(3)
train['target_3'] = df[target]

my_dict3 = {}
for i in range(len(train['target_3'].unique())):
  my_dict3[train['target_3'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict3[target]

In [34]:
def num_to_label3(label):
    label_dict = label_dict = {v: k for k, v in my_dict3.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label3(np.argmax(np.array(pred_3), axis=-1))

test_3['soft_3'] = pd.DataFrame(answer)[1]
test_3['soft_3'] = test_3['soft_3'].astype(int)

# 제출하기

In [35]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [36]:
submission['digit_1'] = test_1['soft_1']

submission['digit_2'] = test_2['soft_2']

submission['digit_3'] = test_3['soft_3']

In [37]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영
3,id_000004,S,30,302,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,A,1,472,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86,862,한의원에서,외래환자위주고,치료
99997,id_099998,G,47,478,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90,902,사업장에서,일반인.학생대상으로,학습공간제공


In [38]:
# submission.to_csv('/content/drive/MyDrive/220407_inference/submission.csv', index=False)